In [1]:
from tqdm import tqdm
import torch.nn as nn
import torch

from blackjackenv_extended import BlackjackEnv
from random_agent import RandomAgent
from basic_strategy_agent import BasicStrategyAgent
from backprop_agent import BackpropAgent
from FFNN_agent import FFNNAgent

In [2]:
KEY = {0: 'STAND', 1:'HIT', 2:'DOUBLE', 3: 'SPLIT'}

rewards = []

def play_game(env, episodes, agent, collect_data=False):

    for episode in tqdm(range(episodes)):
        observation, info = env.reset()
        done = False
        
        # print("hand 1", env.player)
        # print("hand 2", env.player2)
        # print("Start Observation: ", observation)
        
        
        while not done:
            
            action = agent.get_action(observation)
            # print("Action: ", KEY[action])

            # save data for training
            if collect_data:
                agent.collect_data(observation, action)
            
            observation, reward, terminated, truncated, info = env.step(action) 
            # print("hand 1", env.player)
            # print("hand 2", env.player2)
            # print("Observation: ", observation, "Reward: ", reward)
            

            if terminated or truncated:
                # print(f"Dealer hand: ", env.dealer)
                rewards.append(reward)
                observation = env.reset()
                done = True
            

    if collect_data:
        agent.save_data()

In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("using gpu: ", torch.cuda.get_device_name())
else:
    device = torch.device('cpu')
    print("using cpu")

using cpu


In [5]:
env = BlackjackEnv(natural=True)

# Hyperparameters
episodes = 10
collect_data = True


#agent = RandomAgent(env, filename="random_agent")
#agent = BasicStrategyAgent(env, filename="basic_strategy_agent")
# agent = BackpropAgent(env, 
#                       model=torch.load('../models/backprop_model.pth'),
#                       input_size=5, 
#                       output_size=4, 
#                       hidden_size=30, 
#                       activation_fn=nn.ReLU(),
#                       filename="backprop_agent"
#                     )
agent = FFNNAgent(env,
                  model_path='../models/best_model.pth',
                  device=device,
                  layers=[9,30,30],
                  threshold=0.5,
                  lr=0.01,
                  filename="ffnn_agent"
                  )

rewards = []

play_game(env, episodes, agent, collect_data)

average_reward = sum(rewards)/episodes
variance = sum([((x - average_reward) ** 2) for x in rewards]) / (episodes - 1)

#print("Rewards: ", rewards)
print("Average Reward: ", average_reward)
print("Variance: ", variance)

100%|██████████| 10/10 [00:00<00:00, 393.19it/s]

Average Reward:  -0.4
Variance:  2.266666666666667
